In [76]:
from transformers import pipeline
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

In [77]:
data_pd = pd.read_csv('Codage_CV versus HR_for Irene Berezin-Dec-17-2024.csv')

In [78]:
data_pd

,Question,ResponseM,pid
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc
...,...,...,...
877,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc
878,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc
879,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc
880,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc


In [79]:
data_pd.iloc[640]

Question                                        1
ResponseM                                     NaN
pid          25225672-fec2-eb11-a7ad-28187818aecc
Name: 640, dtype: object

In [80]:
non_string_rows = data_pd[~data_pd['ResponseM'].apply(lambda x: isinstance(x, str))]
if not non_string_rows.empty:
    print("Rows with non-string 'ResponseM':")
    print(non_string_rows[['ResponseM']])
else:
    print("All rows in 'ResponseM' are strings.")

Rows with non-string 'ResponseM':
    ResponseM
640       NaN


In [81]:

classifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

def detect_language_transformer(text):
    result = classifier(text[:512], return_all_scores=True)  # Get all probabilities
    scores = {x['label']: x['score'] for x in result[0]}

    target_languages = ['fr', 'en']
    filtered_scores = {k: scores[k] for k in target_languages if k in scores}
    
    # model was labelling some french text as italian, so forcing it to label it as french
    if filtered_scores:
        return max(filtered_scores, key=filtered_scores.get)
    else:
        # Fallback: If neither 'fr' nor 'en' is detected, choose based on closest match
        if 'it' in scores and scores['it'] >= 0.3:
            return 'fr' 
        return 'en' 

data_pd['language'] = data_pd['ResponseM'].dropna().apply(detect_language_transformer)




Device set to use cpu
c:\Users\Irene\miniconda3\envs\fuck\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [82]:
data_pd

,Question,ResponseM,pid,language
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc,en
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc,en
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc,en
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr
...,...,...,...,...
877,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc,en
878,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc,fr
879,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc,en
880,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc,en


In [83]:
french = data_pd[data_pd['language'] == 'fr']
french

,Question,ResponseM,pid,language
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr
5,2,égalité respect liberté famille s’exprimer etr...,25ae2752-eebe-eb11-a7ad-28187818aecc,fr
6,1,famille democracie respecter equitable securit...,a92a5edf-9bb9-eb11-a7ad-28187818aecc,fr
7,1,respect gentillesse honnetete integrite famill...,9f5d9e78-eebe-eb11-a7ad-28187818aecc,fr
...,...,...,...,...
864,2,privilège acquis inégalité société combat espoir,80eccb65-eebe-eb11-a7ad-28187818aecc,fr
870,2,ethnicité egalité esprit de partage différence...,ed215672-fec2-eb11-a7ad-28187818aecc,fr
872,2,bafoués nécessaires charte conventions autocht...,817ad272-9bb9-eb11-a7ad-28187818aecc,fr
874,2,iniquité afrique femmes famine pauvreté égoïsme,1f6191bc-eebe-eb11-a7ad-28187818aecc,fr


In [84]:
label_map = {
    "LABEL_0": "negative",  
    "LABEL_1": "neutral",
    "LABEL_2": "positive",
    "negative": "negative", 
    "neutral":  "neutral",
    "positive": "positive"
}

In [124]:


#preprocessing
english_data = data_pd[data_pd['language'] == 'en']['ResponseM'].astype(str).to_list()
french_data = data_pd[data_pd['language'] == 'fr']['ResponseM'].astype(str).to_list()

batch_size = 32
results_en = []
results_fr = []

def get_sentiment_scores(texts, tokenizer, model, alpha=2.0, batch_size=32):
    all_results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        raw_probs = F.softmax(logits, dim=-1)
        
        adjusted_probs = torch.pow(raw_probs, alpha)
        adjusted_probs = adjusted_probs / adjusted_probs.sum(dim=-1, keepdim=True)


        
        
        for probs in adjusted_probs:
            # Assuming the model output is [neg, neu, pos]
            all_results.append({
                "negative": probs[0].item(),
                "neutral":  probs[1].item(),
                "positive": probs[2].item()
            })
    return all_results


# ----- English Model -----
model_name_en = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer_en = AutoTokenizer.from_pretrained(model_name_en)
model_en = AutoModelForSequenceClassification.from_pretrained(model_name_en)
model_en.eval()  # set model to evaluation mode

model_name_fr = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer_fr = AutoTokenizer.from_pretrained(model_name_fr)
model_fr = AutoModelForSequenceClassification.from_pretrained(model_name_fr)
model_fr.eval()

english_data = data_pd[data_pd['language'] == 'en']['ResponseM'].astype(str).to_list()
french_data  = data_pd[data_pd['language'] == 'fr']['ResponseM'].astype(str).to_list()

results_en = get_sentiment_scores(english_data, tokenizer_en, model_en, batch_size=32)
results_fr = get_sentiment_scores(french_data, tokenizer_fr, model_fr,batch_size=32)




Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [125]:
# Masks to select English or French rows
english_mask = (data_pd['language'] == 'en')
french_mask  = (data_pd['language'] == 'fr')

# 1) Insert English sentiment columns
data_pd.loc[english_mask, 'negative'] = [r["negative"] for r in results_en]
data_pd.loc[english_mask, 'neutral']  = [r["neutral"]  for r in results_en]
data_pd.loc[english_mask, 'positive'] = [r["positive"] for r in results_en]

# 2) Insert French sentiment columns
data_pd.loc[french_mask, 'negative'] = [r["negative"] for r in results_fr]
data_pd.loc[french_mask, 'neutral']  = [r["neutral"]  for r in results_fr]
data_pd.loc[french_mask, 'positive'] = [r["positive"] for r in results_fr]


In [126]:
data_pd.head()


,Question,ResponseM,pid,language,negative,neutral,positive,sentiment,is_uncertain,vader_neg,vader_neu,vader_pos,vader_compound,vader_sentiment
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc,en,0.003980,0.945459,0.050561,neutral,False,0.0,0.755,0.245,0.7579,positive
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc,en,0.002102,0.665008,0.332890,neutral,False,0.0,0.556,0.444,0.8316,positive
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr,0.223104,0.678134,0.098762,neutral,False,0.0,0.543,0.457,0.6369,positive
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc,en,0.001986,0.947480,0.050534,neutral,False,0.0,0.543,0.457,0.6369,positive
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr,0.057707,0.267088,0.675205,positive,False,0.0,0.441,0.559,0.9136,positive


In [127]:
data_pd['sentiment'] = ''

data_pd.loc[(data_pd['negative'] >= data_pd['neutral']) & (data_pd['negative'] >= data_pd['positive']), 'sentiment'] = 'negative'
data_pd.loc[(data_pd['neutral'] > data_pd['negative']) & (data_pd['neutral'] >= data_pd['positive']), 'sentiment'] = 'neutral'
data_pd.loc[(data_pd['positive'] > data_pd['negative']) & (data_pd['positive'] > data_pd['neutral']), 'sentiment'] = 'positive'


In [128]:


import numpy as np

threshold = 0.2

data_pd = data_pd.dropna(subset=['negative', 'neutral', 'positive']).reset_index(drop=True)

def is_uncertain(row, threshold):
    values = [row['negative'], row['neutral'], row['positive']]
    sorted_values = sorted(values, reverse=True)
    return (sorted_values[0] - sorted_values[1]) < threshold

data_pd['is_uncertain'] = data_pd.apply(lambda row: is_uncertain(row, threshold), axis=1)



In [129]:
data_true = data_pd[data_pd['is_uncertain'] == True]
print(len(data_true))

data_false = data_pd
print(len(data_false))

154
881


In [91]:
data_pd

,Question,ResponseM,pid,language,negative,neutral,positive,sentiment,is_uncertain
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc,en,0.000006,0.354325,1.013339e-03,neutral,False
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc,en,0.000001,0.103338,2.589456e-02,neutral,True
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr,0.007406,0.068427,1.451370e-03,neutral,True
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc,en,0.000002,0.376364,1.070630e-03,neutral,False
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr,0.000536,0.011483,7.338931e-02,positive,True
...,...,...,...,...,...,...,...,...,...
876,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc,en,0.140485,0.018821,9.148673e-08,negative,True
877,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc,fr,0.007167,0.114209,2.659440e-04,neutral,True
878,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc,en,0.004106,0.276315,2.273838e-07,neutral,False
879,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc,en,0.090495,0.035146,1.182619e-07,negative,True


In [92]:
print(data_pd.iloc[880].to_dict())

{'Question': 1, 'ResponseM': 'liberalism pro abortion pro gay marriage pro transexual multicultural authoritarian', 'pid': 'fae1254c-eebe-eb11-a7ad-28187818aecc', 'language': 'en', 'negative': 0.07428067922592163, 'neutral': 0.03903684392571449, 'positive': 1.2511811746662715e-06, 'sentiment': 'negative', 'is_uncertain': True}


In [93]:
data_pd.to_csv("sentiment_scores_dataset.csv")

In [94]:
def get_lang(df, lang = str):
    return df[df.language == lang]



In [95]:
import vaderSentiment.vaderSentiment as vs
print(vs.__file__)


c:\Users\Irene\miniconda3\envs\fuck\Lib\site-packages\vaderSentiment\vaderSentiment.py


In [96]:
# Install the English vaderSentiment package normally.
!pip install vaderSentiment

import importlib.util
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as EnglishSentimentIntensityAnalyzer

# Create an analyzer for English.
english_analyzer = EnglishSentimentIntensityAnalyzer()

# Dynamically load the French vader-multi module from its file location.
# Replace '/path/to/vader-multi/vaderSentiment/vaderSentiment.py' with the actual path on your system.
french_module_path = "c:/Users/Irene/miniconda3/envs/fuck/Lib/site-packages/vaderSentiment/vaderSentiment.py"
spec = importlib.util.spec_from_file_location("vaderMulti", french_module_path)
vaderMulti = importlib.util.module_from_spec(spec)
spec.loader.exec_module(vaderMulti)

# Now you have the French SentimentIntensityAnalyzer as a separate name.
FrenchSentimentIntensityAnalyzer = vaderMulti.SentimentIntensityAnalyzer
french_analyzer = FrenchSentimentIntensityAnalyzer()

# Assuming you have a function get_lang() to filter your DataFrame by language.
df_en = get_lang(data_pd, 'en')
df_fr = get_lang(data_pd, 'fr')

def get_sentiment_english(row):
    text = row['ResponseM']
    return english_analyzer.polarity_scores(text)

def get_sentiment_french(row):
    text = row['ResponseM']
    return french_analyzer.polarity_scores(text)

# Apply sentiment analysis: English texts with the English analyzer,
# and French texts with the French analyzer.
data_pd.loc[df_en.index, 'vader'] = df_en.apply(get_sentiment_english, axis=1)
data_pd.loc[df_fr.index, 'vader'] = df_fr.apply(get_sentiment_french, axis=1)

# Expand the dictionary into separate columns.
vader_df = data_pd['vader'].apply(pd.Series)

# Rename columns.
vader_df = vader_df.rename(columns={
    'neg': 'vader_neg',
    'neu': 'vader_neu',
    'pos': 'vader_pos',
    'compound': 'vader_compound'
})

# Join the new columns back to your original DataFrame and drop the 'vader' column.
data_pd = data_pd.join(vader_df).drop(columns=['vader'])
data_pd



,Question,ResponseM,pid,language,negative,neutral,positive,sentiment,is_uncertain,vader_neg,vader_neu,vader_pos,vader_compound
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc,en,0.000006,0.354325,1.013339e-03,neutral,False,0.000,0.755,0.245,0.7579
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc,en,0.000001,0.103338,2.589456e-02,neutral,True,0.000,0.556,0.444,0.8316
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr,0.007406,0.068427,1.451370e-03,neutral,True,0.000,0.543,0.457,0.6369
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc,en,0.000002,0.376364,1.070630e-03,neutral,False,0.000,0.543,0.457,0.6369
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr,0.000536,0.011483,7.338931e-02,positive,True,0.000,0.441,0.559,0.9136
...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc,en,0.140485,0.018821,9.148673e-08,negative,True,0.000,1.000,0.000,0.0000
877,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc,fr,0.007167,0.114209,2.659440e-04,neutral,True,0.000,1.000,0.000,0.0000
878,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc,en,0.004106,0.276315,2.273838e-07,neutral,False,0.000,0.556,0.444,0.8225
879,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc,en,0.090495,0.035146,1.182619e-07,negative,True,0.081,0.702,0.218,0.5994


In [97]:
vader_list = data_pd['vader_compound']
sentiment_values = []
for l in vader_list.to_list(): 
    if l < -0.5:
        sentiment_values.append('negative')
    elif l >= -0.5 and  l <= 0.5:
        sentiment_values.append('neutral')
    else: sentiment_values.append('positive')
    
data_pd['vader_sentiment'] = sentiment_values

data_pd

,Question,ResponseM,pid,language,negative,neutral,positive,sentiment,is_uncertain,vader_neg,vader_neu,vader_pos,vader_compound,vader_sentiment
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc,en,0.000006,0.354325,1.013339e-03,neutral,False,0.000,0.755,0.245,0.7579,positive
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc,en,0.000001,0.103338,2.589456e-02,neutral,True,0.000,0.556,0.444,0.8316,positive
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr,0.007406,0.068427,1.451370e-03,neutral,True,0.000,0.543,0.457,0.6369,positive
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc,en,0.000002,0.376364,1.070630e-03,neutral,False,0.000,0.543,0.457,0.6369,positive
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr,0.000536,0.011483,7.338931e-02,positive,True,0.000,0.441,0.559,0.9136,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc,en,0.140485,0.018821,9.148673e-08,negative,True,0.000,1.000,0.000,0.0000,neutral
877,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc,fr,0.007167,0.114209,2.659440e-04,neutral,True,0.000,1.000,0.000,0.0000,neutral
878,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc,en,0.004106,0.276315,2.273838e-07,neutral,False,0.000,0.556,0.444,0.8225,positive
879,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc,en,0.090495,0.035146,1.182619e-07,negative,True,0.081,0.702,0.218,0.5994,positive


In [98]:
#data_pd.loc[(data_pd['vader_neg'] >= data_pd['vader_neu']) & (data_pd['vader_neg'] >= data_pd['vader_neu']), 'vader_sentiment_alt'] = 'negative'
#data_pd.loc[(data_pd['vader_neu'] > data_pd['vader_neg']) & (data_pd['vader_neu'] >= data_pd['vader_pos']), 'vader_sentiment_alt'] = 'neutral'
#data_pd.loc[(data_pd['vader_pos'] > data_pd['vader_neg']) & (data_pd['vader_pos'] > data_pd['vader_neu']), 'vader_sentiment_alt'] = 'positive'

In [99]:
data_pd.to_csv('sentiment_scored_with_vader.csv', encoding = 'UTF-8')

In [100]:
import pandas as pd

def summary_stat(df):
    # Count values for each sentiment category
    neg_vals = len(df[df.vader_sentiment == 'negative'])
    pos_vals = len(df[df.vader_sentiment == 'positive'])
    neu_vals = len(df[df.vader_sentiment == 'neutral'])
    
    # Total number of entries
    total = len(df)
    
    # Prepare dictionary with values and proportions
    sentiment_summary = {
        'neg': neg_vals,
        'pos': pos_vals,
        'neu': neu_vals,
        'prop_neg': neg_vals / total,
        'prop_pos': pos_vals / total,
        'prop_neu': neu_vals / total
    }
    
    # Return as a DataFrame
    return pd.DataFrame([sentiment_summary])

# Example usage:
summary_stat(data_pd)


,neg,pos,neu,prop_neg,prop_pos,prop_neu
0,48,526,307,0.054484,0.597049,0.348468
